In [48]:
# Celda 1: Importar las librerías necesarias
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate
from tensorflow.keras.preprocessing import image as keras_img
from tqdm import tqdm
import ssl


ssl._create_default_https_context = ssl._create_unverified_context


In [49]:
# Celda 2: Cargar los datos
# Cargar los datasets
product_data = pd.read_csv('../archive/product_data.csv')
attribute_data = pd.read_csv('../archive/attribute_data.csv')
test_data = pd.read_csv('../archive/test_data.csv')


In [50]:
# Celda 3: Fusionar los datasets
# Fusionar los datos de atributos con la información de productos basado en 'cod_modelo_color'
merged_data = pd.merge(attribute_data, product_data, on='cod_modelo_color', how='left')


In [51]:
# Celda 4: Codificar variables categóricas y características
# Codificar variables categóricas
categorical_cols = ['des_sex', 'des_age', 'des_line', 'des_fabric', 'des_product_category',
                    'des_product_aggregated_family', 'des_product_family', 'des_product_type']

for col in categorical_cols:
    le = LabelEncoder()
    merged_data[col] = le.fit_transform(merged_data[col].astype(str))

# Codificar la variable objetivo 'des_value' (para entrenamiento)
le_value = LabelEncoder()
merged_data['des_value_encoded'] = le_value.fit_transform(merged_data['des_value'])
target_classes = len(le_value.classes_)


In [52]:
# Celda 5: Manejar los datos tabulares para el conjunto de prueba
# Codificar los valores de los datos de prueba con los mismos codificadores
for col in categorical_cols:
    test_data[col] = le.fit_transform(test_data[col].astype(str))


In [ ]:
image_dir = "../archive/images/images"
image_files = os.listdir(image_dir)
print(image_files[:5])

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image as keras_img
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np
import os
from tqdm import tqdm

# Inicializar MobileNet para extracción de características
mobilenet = MobileNet(include_top=False, pooling='avg', input_shape=(224, 224, 3))

# Función para extraer características por lotes con manejo de errores
def extract_image_features_batch_mobilenet_with_error_handling(df, image_dir, batch_size=32):
    features = []
    batch = []
    failed_indices = []  # Índices de filas con errores

    for idx, filename in tqdm(enumerate(df['des_filename']), desc="Procesando imágenes", total=len(df)):
        image_path = os.path.join(image_dir, filename)
        try:
            # Cargar y procesar la imagen
            img = keras_img.load_img(image_path, target_size=(160, 224))
            img_data = keras_img.img_to_array(img)
            img_data = keras_img.smart_resize(img_data, (224, 224))  # Redimensionar a 224x224
            img_data = preprocess_input(img_data)  # Preprocesamiento de MobileNet
            batch.append(img_data)
        except Exception as e:
            print(f"Error procesando imagen {filename}: {e}")
            failed_indices.append(idx)  # Guardar el índice de la fila con error
            continue

        # Si el lote alcanza el tamaño definido
        if len(batch) == batch_size:
            try:
                batch_array = np.array(batch)
                batch_features = mobilenet.predict(batch_array)  # Predicción en lote
                features.extend(batch_features)
                batch = []  # Reiniciar el lote
            except Exception as e:
                print(f"Error procesando lote: {e}")
                continue

    # Procesar el último lote si no está vacío
    if batch:
        try:
            batch_array = np.array(batch)
            batch_features = mobilenet.predict(batch_array)
            features.extend(batch_features)
        except Exception as e:
            print(f"Error procesando lote final: {e}")

    # Eliminar las filas correspondientes a imágenes fallidas
    df.drop(index=failed_indices, inplace=True)
    return np.array(features)

# Rutas de imágenes y nombres
image_dir = "../archive/images/images"  

# Generar características para entrenamiento
train_image_features = extract_image_features_batch_mobilenet_with_error_handling(
    merged_data, image_dir, batch_size=32
)

# Generar características para prueba
test_image_features = extract_image_features_batch_mobilenet_with_error_handling(
    test_data, image_dir, batch_size=32
)

# Guardar características para uso futuro
np.save('train_image_features_mobilenet.npy', train_image_features)
np.save('test_image_features_mobilenet.npy', test_image_features)



In [ ]:


# Cargar características
train_image_features = np.load('train_image_features.npy')
test_image_features = np.load('test_image_features.npy')


In [ ]:
# Celda 7: Combinar características tabulares e imágenes
# Combinar características tabulares con características de imagen para el conjunto de entrenamiento
X_train_tabular = merged_data[categorical_cols].values
X_train_image = train_image_features

# Para el conjunto de prueba
X_test_tabular = test_data[categorical_cols].values
X_test_image = test_image_features

# Estandarizar las características tabulares (escalado opcional)
scaler = StandardScaler()
X_train_tabular = scaler.fit_transform(X_train_tabular)
X_test_tabular = scaler.transform(X_test_tabular)

# Combinar las características (tabulares + imagen)
def combine_features(tabular_data, image_data):
    return [tabular_data, image_data]

X_train = combine_features(X_train_tabular, X_train_image)
X_test = combine_features(X_test_tabular, X_test_image)

# Variable objetivo
y_train = to_categorical(merged_data['des_value_encoded'], num_classes=target_classes)


In [ ]:
# Celda 8: Desarrollo del modelo usando redes neuronales y ResNet
# Definir la forma de los datos tabulares e imagen
tabular_input = Input(shape=(len(categorical_cols),))
image_input = Input(shape=(2048,))  # Tamaño de las características de ResNet (pooling='avg' da 2048 dim)

# Rama de datos tabulares
x1 = Dense(256, activation='relu')(tabular_input)
x1 = Dense(128, activation='relu')(x1)

# Rama de datos de imagen
x2 = Dense(256, activation='relu')(image_input)
x2 = Dense(128, activation='relu')(x2)

# Combinar las ramas
combined = concatenate([x1, x2])
combined = Dense(128, activation='relu')(combined)
combined = Dense(64, activation='relu')(combined)

# Capa de salida para clasificación multiclase
output = Dense(target_classes, activation='softmax')(combined)

# Crear el modelo
model = Model(inputs=[tabular_input, image_input], outputs=output)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mostrar resumen del modelo
model.summary()


In [ ]:
# Celda 9: Entrenamiento del modelo
# Entrenar el modelo utilizando una partición para validación
X_com = combine_features(X_train_tabular, X_train_image)
model.fit(X_com, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Celda 10: Predicción y Generación de la Subida
# Predecir sobre el conjunto de prueba
X_test_com = combine_features(X_test_tabular, X_test_image)
predictions = model.predict(X_test_com)

# Decodificar las predicciones
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = le_value.inverse_transform(predicted_classes)

# Crear archivo de sumisión
submission = pd.DataFrame({
    'test_id': test_data['test_id'],
    'des_value': predicted_labels
})

# Guardar archivo de sumisión a CSV
submission.to_csv('submission.csv', index=False)


In [ ]:
# Celda 11: Manejo de predicciones INVÁLIDAS
# Si es necesario, puedes volver a ejecutar una función de aplicabilidad que verifique si un atributo es válido para ciertos tipos de productos
def mark_as_invalid(df, rules):
    for idx, row in df.iterrows():
        if not rule_applies(row['attribute_name'], row['des_product_type']):  # Función personalizada de reglas
            df.at[idx, 'des_value'] = 'INVALID'
    return df

# Función de regla personalizada para reemplazar los inválidos
def rule_applies(attribute, product_type):
    # Definir las reglas de aplicabilidad aquí
    if attribute == 'heel_shape_type' and product_type not in ['Shoes', 'Sandals', 'Footwear']:
        return False
    return True

# Aplicarlo
submission = mark_as_invalid(submission, rules={})
